# Student's t-test for significantly different means in Python

In [1]:
import numpy as np
import timeit
import sys
import os
from subprocess import call, check_output


owd = os.getcwd()  # save for later

# Import Python functions
sys.path.append(os.path.realpath('../../avevar'))
from avevar import avevar

sys.path.append(os.path.realpath('../../../Ch6_SpecialFunctions/betai'))
from betai import betai

# Compile Fortran source code (using a Python script)
os.system('python compile_fortran.py')

os.chdir(owd)

Data input:

In [2]:
data1 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
data2 = np.array([1.1, 2, 3, 4, 5, 6, 7, 8, 9])

Function definition:

In [3]:
def ttest(data1, data2):
    # Given the arrays "data1" and "data2", returns the
    # Student's "t" and its significance as "prob".
    # Data are assumed to be drawn from populations with
    # the same true variance.
    # Small values of "prob" indicates that the arrays 
    # have different means.
    n1 = len(data1)
    n2 = len(data2)
    
    ave1, var1 = avevar(data1)
    ave2, var2 = avevar(data2)

    # print(ave1, ave2)
    # print(var1, var2)

    # degrees of freedom
    df = n1 + n2 - 2.
    # pooled variance
    var = ((n1 - 1.) * var1 + (n2 - 1) * var2) / df
    # Student's t
    t = (ave1 - ave2) / np.sqrt(var * (1/n1 + 1/n2))
    # significance
    prob = betai(0.5 * df, 0.5, df/(df + t**2))
    
    return t, prob

#### Calling the Python ttest function:

In [4]:
t, prob = ttest(data1, data2)

print(t, prob)

-0.008635140708713388 0.9932169925849851


#### Calling the Fortran ttest function:

In [5]:
fortranOutput = check_output('./fmain_ttest').decode("utf-8").replace('\n','').split(' ')
[float(i) for i in fortranOutput if i]

[-0.008635140708713388, 0.9932169925849851]

# Timing against Fortran:

Test function in **Python** and timing:

In [6]:
def speed_test():
    for i in range(1000):
        ttest(data1, data2)
        
%timeit speed_test()

15.4 ms ± 285 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Timing in **Fortran**:

In [7]:
%timeit call(['./fspeed_ttest'])

5.35 ms ± 59.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
